In [5]:
import heapq
import numpy as np

# Formato de entrada: propuestas con usuarios apoyos ("U...") y rechazos ("R...")
data = {
    "Prop1": ["U2", "U1", "U3", "U6", "R4", "R5"],
    "Prop2": ["U2", "U4", "U6", "R1", "R3"],
    "Prop3": ["U1", "U5", "U7", "U8", "U9", "U11", "R6", "R2"],
    "Prop4": ["U3", "U8", "R9", "U9", "R7", "U10", "U11", "U12", "R1"],
}
# Lista ordenada de propuestas (coincide con filas/columnas de closeness_matrix)
proposals = list(data.keys())
# Matriz de cercanía Propuestas x Propuestas (orden: proposals)
closeness_matrix = np.array([
    [1.0, 0.9, 0.9, 0.1],
    [0.9, 1.0, 0.9, 0.1],
    [0.9, 0.9, 1.0, 0.1],
    [0.1, 0.1, 0.1, 1.0],
])

# Extraer usuarios y propuestas que apoyan
user_set = set()
user_supported = {}
for idx, prop in enumerate(proposals):
    for entry in data[prop]:
        if entry.startswith('U'):
            user = entry
        elif entry.startswith('R'):
            user = 'U' + entry[1:]
        else:
            continue
        user_set.add(user)
        if entry.startswith('U'):
            user_supported.setdefault(user, []).append(idx)
# Lista ordenada de usuarios
user_names = sorted(user_set, key=lambda x: int(x[1:]))

# Determinar propuesta objetivo: la de mayor apoyo inicial
support_counts = {prop: sum(1 for e in data[prop] if e.startswith('U')) for prop in proposals}
goal_prop = max(support_counts, key=support_counts.get)
goal_idx = proposals.index(goal_prop)

# Calcular cercanía de anclaje por usuario\# Si el usuario apoyaba otras propuestas, anchoring = max closeness(goal, esas)
anchor_closeness = {}
for user in user_names:
    idxs = user_supported.get(user, [])
    if idxs:
        anchor_closeness[user] = max(closeness_matrix[goal_idx][i] for i in idxs)
    else:
        anchor_closeness[user] = 0.0

# Construir estado inicial y objetivo
init_list = data[goal_prop]
supporters = {e for e in init_list if e.startswith('U')}
rejecters = {'U' + e[1:] for e in init_list if e.startswith('R')}
init_state = tuple(
    1 if u in supporters else -1 if u in rejecters else 0
    for u in user_names
)
target_state = tuple(1 for _ in user_names)

# Generar estados vecinos: un solo usuario cambia -1->0 o 0->1
def neighbors(state):
    for i, v in enumerate(state):
        if v == -1:
            new = list(state); new[i] = 0
            yield tuple(new), i, -1, 0
        if v == 0:
            new = list(state); new[i] = 1
            yield tuple(new), i, 0, 1

# Búsqueda Dijkstra con heapq, calculando coste según cercanía
def dijkstra_states(start, target):
    heap = [(0.0, start, [start], [], [])]  # cost, state, path, actions, step_costs
    best_cost = {start: 0.0}
    while heap:
        cost, state, path, actions, step_costs = heapq.heappop(heap)
        if state == target:
            return cost, path, actions, step_costs
        if cost > best_cost[state]:
            continue
        for neigh, idx, old, new in neighbors(state):
            user = user_names[idx]
            # Coste del paso = inverso de la cercanía anclada
            c = anchor_closeness[user]
            step_cost = 1.0 / (c + 0.01)
            new_cost = cost + step_cost
            if neigh not in best_cost or new_cost < best_cost[neigh]:
                best_cost[neigh] = new_cost
                # Descripción de la acción incluyendo propuesta
                if old == -1 and new == 0:
                    act = f"{user}: deja de rechazar {goal_prop}"
                elif old == 0 and new == 1:
                    act = f"{user}: apoya {goal_prop}"
                else:
                    act = f"{user}: {old}->{new} {goal_prop}"
                heapq.heappush(heap, (new_cost, neigh,
                                      path + [neigh],
                                      actions + [act],
                                      step_costs + [step_cost]))
    return None

if __name__ == '__main__':
    result = dijkstra_states(init_state, target_state)
    if result is None:
        print("No se encontró camino al consenso completo.")
    else:
        total_cost, states_path, actions, step_costs = result
        print(f"Camino encontrado con costo total: {total_cost:.2f}\n")
        # Mostrar cada paso, U# primero, luego R#
        for idx, (act, sc) in enumerate(zip(actions, step_costs), start=1):
            state = states_path[idx]
            supporters = [u for i, u in enumerate(user_names) if state[i] == 1]
            rejects   = ['R' + u[1:] for i, u in enumerate(user_names) if state[i] == -1]
            markers = supporters + rejects
            print(f"Paso {idx} (coste: {sc:.2f}): acción = {act}")
            print(f"  Estado: {', '.join(markers)}")
        print(f"\nPropuesta final de consenso: {goal_prop}")



Camino encontrado con costo total: 24.78

Paso 1 (coste: 1.10): acción = U6: deja de rechazar Prop3
  Estado: U1, U5, U7, U8, U9, U11, R2
Paso 2 (coste: 1.10): acción = U6: apoya Prop3
  Estado: U1, U5, U6, U7, U8, U9, U11, R2
Paso 3 (coste: 9.09): acción = U12: apoya Prop3
  Estado: U1, U5, U6, U7, U8, U9, U11, U12, R2
Paso 4 (coste: 1.10): acción = U4: apoya Prop3
  Estado: U1, U4, U5, U6, U7, U8, U9, U11, U12, R2
Paso 5 (coste: 1.10): acción = U3: apoya Prop3
  Estado: U1, U3, U4, U5, U6, U7, U8, U9, U11, U12, R2
Paso 6 (coste: 1.10): acción = U2: deja de rechazar Prop3
  Estado: U1, U3, U4, U5, U6, U7, U8, U9, U11, U12
Paso 7 (coste: 1.10): acción = U2: apoya Prop3
  Estado: U1, U2, U3, U4, U5, U6, U7, U8, U9, U11, U12
Paso 8 (coste: 9.09): acción = U10: apoya Prop3
  Estado: U1, U2, U3, U4, U5, U6, U7, U8, U9, U10, U11, U12

Propuesta final de consenso: Prop3
